<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
       Importando bibliotecas
    </strong>
</p>

In [8]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as ws
from datetime import datetime

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Criando planilha de controle de entregas
    </strong>
</p>

In [9]:
controle_path = 'dados/controle_entregas.xlsx'

wb = Workbook()

ws = wb.active

ws.title = 'Entregas'

ws.append(['Data' ,
'Estado' ,
'Motorista' ,
'Entregas Previstas' ,
'Entregas no Prazo' ,
'Entregas Realizadas'])

ws.append(['01/01/2026','SP','Carlos',10, 6, 8])
ws.append(['03/01/2026','RJ','Pedro',10, 7, 9])
ws.append(['05/01/2026','MG','Luiz',10, 5, 10])
ws.append(['10/01/2026','SP','João',10, 8, 9])
ws.append(['02/02/2026','SP','Paulo',10, 4, 9])

wb.save(controle_path)

print('Planilha criada com sucesso!')

Planilha criada com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Inserindo novos indicadores
    </strong>
</p>

In [10]:
wb = load_workbook(controle_path)
ws = wb.active

# criando colunas
ws["G1"] = "Taxa de Sucesso" 
ws["H1"] = "Taxa de Atraso"

for linha in range(2, ws.max_row + 1):
    ws[f'G{linha}'] = f'=F{linha} / D{linha}'
    ws[f'H{linha}'] = f'= 1 - G{linha}'

wb.save(controle_path)
print('Arquivo atualizado com sucesso!')

Arquivo atualizado com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Formatando as taxas como percentual
    </strong>
</p>

In [11]:
for linha in range(2, ws.max_row + 1):
    ws[f'G{linha}'].number_format = "0.00%"
    ws[f'H{linha}'].number_format = "0.00%"

wb.save(controle_path)
print('Arquivo atualizado com sucesso!')

Arquivo atualizado com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Resumo gerencial por estado
    </strong>
</p>

In [ ]:

df = pd.read_excel(controle_path, sheet_name="Entregas")

df['Taxa de Sucesso 2'] = df['Entregas Realizadas'] / df['Entregas Previstas']
df['Taxa de Atraso 2'] = 1 - df['Taxa de Sucesso 2']

resumo_estado = df.groupby('Estado').agg(
    Total_Entregas_Previstas = ('Entregas Previstas', 'sum'),
    Total_Entregas_Realizadas = ('Entregas Realizadas', 'sum'),
    Total_Entregas_no_Prazo = ('Entregas no Prazo', 'sum'),
    Taxa_de_Sucesso = ('Taxa de Sucesso 2', 'mean'),
    Taxa_de_Atraso = ('Taxa de Atraso 2', 'mean')
)

wb = load_workbook(controle_path)

if 'Resumo' in wb.sheetnames:
    del wb['Resumo']

ws_resumo = wb.create_sheet('Resumo')

ws_resumo.append([
    "Estado",
    "Total_Entregas_Previstas",
    "Total_Entregas_Realizadas",
    "Total_Entregas_no_Prazo",
    "Taxa_de_Sucesso",
    "Taxa_de_Atraso"
])

for estado, valores in resumo_estado.iterrows(): # lista = enumerate / dict = .item() / planilha = iterrows()
    ws_resumo.append([
        estado,
        valores['Total_Entregas_Previstas'],
        valores['Total_Entregas_Realizadas'],
        valores['Total_Entregas_no_Prazo'],
        valores['Taxa_de_Sucesso'],
        valores['Taxa_de_Atraso']
    ])

for linha in range(2, ws_resumo.max_row + 1):
    ws_resumo[f'F{linha}'].number_format = '0.00%'
    ws_resumo[f'E{linha}'].number_format = '0.00%'

wb.save(controle_path)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Cálculos de taxas e inserindo no excel
    </strong>
</p>